<a href="https://colab.research.google.com/github/LC1332/Luotuo-Text-Embedding/blob/main/notebook/TestingVisualizeFeature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dolly Feature Visualization

这个notebook是Luotuo-Testing项目中的一个实验

在陈舒年抽取特征之后

由李鲁鲁进行可视化，

- [x] 用tSNE可视化Dolly数据集上，Lamini和Mini骆驼的特征
- [x] 可以尝试将LaMini Mapping到迷你骆驼，看看特征在不在一起
- [x] 反过来将迷你骆驼 Mapping到Lamini，看看特征在不在一起
- [x] 因为陈舒年正好还误抽了迷你骆驼用英文下的特征，照理说是个很弱的baseline，可以同步做一下对比

## 载入数据

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls /content/drive/MyDrive/GPTData/Testing

code				   LaMini-GPT-124M-Dolly_en.pkl
GPT2-3.5B-Dolly_zh.pkl		   WizardTrain_LaMini-124_en.pkl
GPT2-LuotuoMini-3.5B-Dolly_en.pkl  WizardTrain_LaMini-GPT-1.5B_en.pkl


其中

LaMini-GPT-124M-Dolly_en.pkl 是mini骆驼的一个小特征

GPT2-3.5B-Dolly_zh.pkl		   是迷你骆驼的特征 中文，正确的

GPT2-LuotuoMini-3.5B-Dolly_en.pkl 照理说要弱于前者

让GPT老师干活

```
在 /content/drive/MyDrive/GPTData/Testing 目录下有三个pkl文件

分别对LaMini-GPT-124M-Dolly_en.pkl , GPT2-3.5B-Dolly_zh.pkl		   
GPT2-LuotuoMini-3.5B-Dolly_en.pkl进行读取

存储到lamini, luotuo_zh, luotuo35_en
```

In [19]:
import pickle

with open('/content/drive/MyDrive/GPTData/Testing/LaMini-GPT-124M-Dolly_en.pkl', 'rb') as f:
    lamini = pickle.load(f)
    lamini = lamini['feature']
with open('/content/drive/MyDrive/GPTData/Testing/GPT2-LuotuoMini-3.5B-Dolly_en.pkl', 'rb') as f:
    luotuo35_en = pickle.load(f)
    luotuo35_en = luotuo35_en['feature']
with open('/content/drive/MyDrive/GPTData/Testing/GPT2-3.5B-Dolly_zh.pkl', 'rb') as f:
    luotuo35_zh = pickle.load(f)
    luotuo35_zh = luotuo35_zh['feature']

In [11]:
print(luotuo_zh.shape)

torch.Size([14962, 3072])


## 载入Dolly数据

In [13]:
!pip install -q datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 103.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.8 MB/s eta 0:00:00


In [14]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split

dataset = load_dataset("silk-road/chinese-dolly-15k")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/silk-road___json/silk-road--chinese-dolly-15k-a9aca306f7816495/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
print(dataset['train'])
print(lamini.shape)
print(luotuo35_en.shape)
print(luotuo35_zh.shape)
print(dataset['train'][0].keys())

Dataset({
    features: ['id', 'category', 'instruction_zh', 'context_zh', 'response', 'instruction', 'context'],
    num_rows: 15011
})
torch.Size([14962, 768])
torch.Size([14962, 3072])
torch.Size([14962, 3072])
dict_keys(['id', 'category', 'instruction_zh', 'context_zh', 'response', 'instruction', 'context'])


这个时候我想起来因为最后49条没有完成翻译，所以被舍去了。

这里我们初步对文本进行一下处理

In [23]:
category = [ data['category'] for data in dataset['train'] ]
instruction = [ data['instruction_zh'] for data in dataset['train'] ]

In [25]:
category = category[:-49]
instruction = instruction[:-49]
print(len(category))

14962


## 可视化的初步尝试

这里我不知道陈舒年的代码是否直接可用

我们斗胆来直接用一下

In [26]:
!pip -q install openTSNE

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 56.8 MB/s eta 0:00:00


In [27]:
!git clone https://github.com/LC1332/Luotuo-Text-Embedding.git

Cloning into 'Luotuo-Text-Embedding'...
remote: Enumerating objects: 612, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 612 (delta 1), reused 5 (delta 1), pack-reused 602
Receiving objects: 100% (612/612), 36.28 MiB | 26.16 MiB/s, done.
Resolving deltas: 100% (366/366), done.


In [28]:
%cd Luotuo-Text-Embedding

/content/Luotuo-Text-Embedding


In [36]:
import sys
sys.path.append("..")
from lib.tsne import TSNE_Plot

feature = luotuo35_zh

n = 2000

sub_ins = instruction[:n]
sub_ebd = feature[:n, :]
sub_lab = category[:n]

tsne_plot = TSNE_Plot(sub_ins , sub_ebd, label = sub_lab, n_annotation_positions = 0)
tsne_plot.tsne_plot(n_sentence=40)

让我们再画一下Lamini 120M的

In [37]:

feature = lamini

n = 2000

sub_ins = instruction[:n]
sub_ebd = feature[:n, :]
sub_lab = category[:n]

tsne_plot = TSNE_Plot(sub_ins , sub_ebd, label = sub_lab, n_annotation_positions = 0)
tsne_plot.tsne_plot(n_sentence=40)

直观上来看，Lamini124M的空间就没有3.5B 的迷你骆驼要好了。让我们再去看一看。陈舒年抽错的英语的3.5B的空间。

In [38]:
feature = luotuo35_en

n = 2000

sub_ins = instruction[:n]
sub_ebd = feature[:n, :]
sub_lab = category[:n]

tsne_plot = TSNE_Plot(sub_ins , sub_ebd, label = sub_lab, n_annotation_positions = 0)
tsne_plot.tsne_plot(n_sentence=40)

可以看到这个是不太好的，

那我就在想了，反正刚才的实验是用了前2000来做

那我们不如就拿前2000来作为测试集，然后2000-15000的就作为训练集，

去求闭式解

## 训练闭式解

让GPT老师来给我们做吧

```
实现一段用Python求解岭回归的函数rigid_regress，给定两个numpy的输入X和Y和约束系数lambda

比如 X矩阵是 torch.Size([10000, 3072])
Y矩阵是 torch.Size([10000, 768])

返回矩阵A，使得 X * A 尽可能接近Y

即 A = inv(X' X  + lambda I ) * X' Y

并且告诉我lambda系数大致的确定方法

同时，实现一个测试函数 predict(X,A)，可以返回估计的Y
```

先为GPT老师准备一下数据

In [39]:
n = 2000

lamini_train = lamini[n:,:]
luotuo35_en_train = luotuo35_en[n:,:]
luotuo35_zh_train = luotuo35_zh[n:,:]

然后贴上G老师的函数

In [42]:
import numpy as np

# 定义岭回归函数
def rigid_regress(X, Y, lam = 5):
    XTX_plus_lamI = np.dot(np.transpose(X), X) + lam * np.identity(X.shape[1])
    XTY = np.dot(np.transpose(X), Y)
    A = np.dot(np.linalg.inv(XTX_plus_lamI), XTY)
    return A

# 测试函数
def predict(X, A):
    Y_pred = np.dot(X, A)
    return Y_pred

In [43]:
A_35zh_2_lamini = rigid_regress( luotuo35_zh_train, lamini_train)
print(A_35zh_2_lamini.shape)

(3072, 768)


In [44]:
lamini_test = lamini[:n,:]
luotuo35_en_test = luotuo35_en[:n,:]
luotuo35_zh_test = luotuo35_zh[:n,:]

In [45]:
lamini_by_luotuo35zh = predict( luotuo35_zh_test,  A_35zh_2_lamini)
print(lamini_by_luotuo35zh.shape)

(2000, 768)


可以可以，测试基本通过了，让我们一鼓作气把六组关系都给做了吧

让GPT老师替我们来做一下


```python
lamini_train = lamini[n:,:]
luotuo35_en_train = luotuo35_en[n:,:]
luotuo35_zh_train = luotuo35_zh[n:,:]

lamini_test = lamini[:n,:]
luotuo35_en_test = luotuo35_en[:n,:]
luotuo35_zh_test = luotuo35_zh[:n,:]

A_luotuo35zh_2_lamini = rigid_regress( luotuo35_zh_train, lamini_train)
lamini_by_luotuo35zh = predict( luotuo35_zh_test,  A_luotuo35zh_2_lamini)
print('lamini_by_luotuo35zh','done')

续写这段代码，继续生成 lamini_by_luotuo35en, luotuo35zh_by_lamini,luotuo35zh_by_luotuo35en, luotuo35en_by_luotuo35zh
```




In [46]:
# 通过 luotuo35_zh 预测 lamini
A_luotuo35zh_2_lamini = rigid_regress(luotuo35_zh_train, lamini_train, 0.1)
lamini_by_luotuo35zh = predict(luotuo35_zh_test, A_luotuo35zh_2_lamini)
print('lamini_by_luotuo35zh', 'done', lamini_by_luotuo35zh.shape)

# 通过 luotuo35_en 预测 lamini
A_luotuo35en_2_lamini = rigid_regress(luotuo35_en_train, lamini_train, 0.1)
lamini_by_luotuo35en = predict(luotuo35_en_test, A_luotuo35en_2_lamini)
print('lamini_by_luotuo35en', 'done', lamini_by_luotuo35en.shape)

# 通过 lamini 预测 luotuo35_zh
A_lamini_2_luotuo35zh = rigid_regress(lamini_train, luotuo35_zh_train, 0.1)
luotuo35zh_by_lamini = predict(lamini_test, A_lamini_2_luotuo35zh)
print('luotuo35zh_by_lamini', 'done', luotuo35zh_by_lamini.shape)

# 通过 luotuo35_en 预测 luotuo35_zh
A_luotuo35en_2_luotuo35zh = rigid_regress(luotuo35_en_train, luotuo35_zh_train, 0.1)
luotuo35zh_by_luotuo35en = predict(luotuo35_en_test, A_luotuo35en_2_luotuo35zh)
print('luotuo35zh_by_luotuo35en', 'done', luotuo35zh_by_luotuo35en.shape)

# 通过 luotuo35_zh 预测 luotuo35_en
A_luotuo35zh_2_luotuo35en = rigid_regress(luotuo35_zh_train, luotuo35_en_train, 0.1)
luotuo35en_by_luotuo35zh = predict(luotuo35_zh_test, A_luotuo35zh_2_luotuo35en)
print('luotuo35en_by_luotuo35zh', 'done', luotuo35en_by_luotuo35zh.shape)

lamini_by_luotuo35zh done (2000, 768)
lamini_by_luotuo35en done (2000, 768)
luotuo35zh_by_lamini done (2000, 3072)
luotuo35zh_by_luotuo35en done (2000, 3072)
luotuo35en_by_luotuo35zh done (2000, 3072)


## 用T-SNE初步检验回归的结果

这里我们肯定是想把两组特征画在一起

让我们来修改一下t-SNE那段代码

把luotuo35_zh 和luotuo35zh_by_lamini 画在同一个空间

让我们先把数据组织一下

In [59]:
import torch
n = 2000
feature = luotuo35_zh[:n,:]
feature_est = torch.from_numpy(luotuo35zh_by_lamini[:n,:])

merged_feat = torch.cat((feature, feature_est), dim=0)
print(merged_feat.shape)

# 对于 est的 category，我们统一加上_trans的识别符

sub_lab = category[:n]
sub_lab_trans = [lab+'_trans' for lab in sub_lab]
merged_lab = sub_lab + sub_lab_trans

sub_ins = instruction[:n]
merged_ins = sub_ins + sub_ins


torch.Size([4000, 3072])


```
merged_lab 是一个 长度为4000 的list of string
merged_ins 是一个 长度为4000 的list of string
merged_feat 是一个 4000 * 3072 的torch 矩阵
三者的数据是一一对应的

我希望将三者根据merged_lab的字典升序进行重新排序

请用python为我实现

```

In [62]:
# 使用zip函数将三个列表合并成一个元素为元组的列表
lst = list(zip(merged_lab, merged_ins, merged_feat))

# 对lst进行排序，以merged_lab为关键字
sorted_lst = sorted(lst, key=lambda x: x[0])

# 使用zip(*lst)将排序后的列表重新拆分成三个列表
sorted_merged_lab, sorted_merged_ins, sorted_merged_feat = zip(*sorted_lst)
sorted_merged_feat = torch.stack(sorted_merged_feat)


In [63]:
tsne_plot = TSNE_Plot(sorted_merged_ins , sorted_merged_feat, label = sorted_merged_lab)
tsne_plot.tsne_plot(n_sentence=30)

看来画4000个点有点慢，我们把数据降到1000*2，然后看一下3.5B拟合lamini的情况

In [65]:
import torch
n = 1000
feature = lamini[:n,:]
feature_est = torch.from_numpy(lamini_by_luotuo35zh[:n,:])

merged_feat = torch.cat((feature, feature_est), dim=0)
print(merged_feat.shape)

# 对于 est的 category，我们统一加上_trans的识别符

sub_lab = category[:n]
sub_lab_trans = [lab+'_trans' for lab in sub_lab]
merged_lab = sub_lab + sub_lab_trans

sub_ins = instruction[:n]
merged_ins = sub_ins + sub_ins

# 使用zip函数将三个列表合并成一个元素为元组的列表
lst = list(zip(merged_lab, merged_ins, merged_feat))

# 对lst进行排序，以merged_lab为关键字
sorted_lst = sorted(lst, key=lambda x: x[0])

# 使用zip(*lst)将排序后的列表重新拆分成三个列表
sorted_merged_lab, sorted_merged_ins, sorted_merged_feat = zip(*sorted_lst)
sorted_merged_feat = torch.stack(sorted_merged_feat)


tsne_plot = TSNE_Plot(sorted_merged_ins , sorted_merged_feat, label = sorted_merged_lab)
tsne_plot.tsne_plot(n_sentence=30)

torch.Size([2000, 768])


这两个其实都还可以，让我们用一个差一点的特征去拟合看看

In [66]:
import torch
n = 1000
feature = lamini[:n,:]
feature_est = torch.from_numpy(lamini_by_luotuo35en[:n,:])

merged_feat = torch.cat((feature, feature_est), dim=0)
print(merged_feat.shape)

# 对于 est的 category，我们统一加上_trans的识别符

sub_lab = category[:n]
sub_lab_trans = [lab+'_trans' for lab in sub_lab]
merged_lab = sub_lab + sub_lab_trans

sub_ins = instruction[:n]
merged_ins = sub_ins + sub_ins

# 使用zip函数将三个列表合并成一个元素为元组的列表
lst = list(zip(merged_lab, merged_ins, merged_feat))

# 对lst进行排序，以merged_lab为关键字
sorted_lst = sorted(lst, key=lambda x: x[0])

# 使用zip(*lst)将排序后的列表重新拆分成三个列表
sorted_merged_lab, sorted_merged_ins, sorted_merged_feat = zip(*sorted_lst)
sorted_merged_feat = torch.stack(sorted_merged_feat)


tsne_plot = TSNE_Plot(sorted_merged_ins , sorted_merged_feat, label = sorted_merged_lab)
tsne_plot.tsne_plot(n_sentence=30)

torch.Size([2000, 768])


In [68]:
import torch
n = 1000
feature = luotuo35_zh[:n,:]
feature_est = torch.from_numpy(luotuo35zh_by_luotuo35en[:n,:])

merged_feat = torch.cat((feature, feature_est), dim=0)
print(merged_feat.shape)

# 对于 est的 category，我们统一加上_trans的识别符

sub_lab = category[:n]
sub_lab_trans = [lab+'_trans' for lab in sub_lab]
merged_lab = sub_lab + sub_lab_trans

sub_ins = instruction[:n]
merged_ins = sub_ins + sub_ins

# 使用zip函数将三个列表合并成一个元素为元组的列表
lst = list(zip(merged_lab, merged_ins, merged_feat))

# 对lst进行排序，以merged_lab为关键字
sorted_lst = sorted(lst, key=lambda x: x[0])

# 使用zip(*lst)将排序后的列表重新拆分成三个列表
sorted_merged_lab, sorted_merged_ins, sorted_merged_feat = zip(*sorted_lst)
sorted_merged_feat = torch.stack(sorted_merged_feat)


tsne_plot = TSNE_Plot(sorted_merged_ins , sorted_merged_feat, label = sorted_merged_lab)
tsne_plot.tsne_plot(n_sentence=30)

torch.Size([2000, 3072])


其实这张图是能看到_trans的特征被甩出来的现象的。